In [1]:
########################################   SO      #####################################
### Tratamiento de datos
import glob
import os
import pandas as pd
import numpy as np
import calendar
from contextlib import contextmanager
from datetime import datetime
import math
import warnings
warnings.filterwarnings("ignore")

# E0 # Lectura ####################

os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\02 SELL\Sell OUT\Cierre')
SO = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep='\t',dtype={'Vol HL': np.float64} ,decimal=',')#,dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')
    SO.append(data)
SO = pd.concat(SO,ignore_index=True)

# E1 # Transformacion de la Base  #######

SO.rename(columns = {'Cód Cliente KO':'C_Cli','CD Local':'CD','SKU KO':'Sku_KO','N.Planilla':'Planilla','Canal ABI':'Seg','Vol cj físicas':'CJ','Vol HL':'Hl'} ,inplace=True)
SO['Planilla'] = SO.Fecha + SO.C_Cli.astype(str)
DF =SO
SO = SO[['Fecha','Año','Mes','CD','C_Cli','Planilla','Seg','Sku_KO','CJ','Hl','Bottler']]
Titulo = SO[["Fecha"]]
Titulo = Titulo["Fecha"].str.split("-", n=2, expand=True)
SO['Fecha_f'] = Titulo[2].astype(str)+Titulo[1].astype(str)+Titulo[0].astype(str)
SO['Fecha_f'] = pd.to_datetime(SO.Fecha_f, format="%Y%m%d")
SO['A_Mes'] = Titulo[2].astype(str)+Titulo[1].astype(str)

# E2 # Validacion de datos Por cada campo

# E2 C1:HL 
SO.Hl = SO.Hl.astype(float)
SO.Hl = SO.Hl.round(3)
SO.Hl = SO.Hl*1000
SO.Hl = SO.Hl.round(0)

SO.CJ = SO.CJ.astype(float)
SO.CJ = SO.CJ.round(3)
SO.CJ = SO.CJ*1000
SO.CJ = SO.CJ.round(0)

# E2 C2:CD
SO["CD"] = SO.CD.str.replace(r"Depósito" ,"")
SO["CD"] = SO.CD.str.replace(r"Planta ","")
SO["CD"] = SO.CD.str.replace(r"ú","u")
SO["CD"] = SO.CD.str.replace(r"ó","o")
SO["CD"] = SO.CD.str.replace(r"ñ","n")
SO["CD"] = SO.CD.str.replace(r"Villarica","Villarrica")
SO["CD"] = SO.CD.str.replace(r" ","")
SO["CD"] = SO.CD.str.lower()
#SO = SO.dropna(subset=['Sku_KO','CD','Seg','Fecha_f','A_Mes','Hl'])

# E2 C3:SKU
###### MAESTRAS ####
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Maestras')
Maestro_SKU = pd.read_excel('Nueva mastra SKU_PEPO.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','SKU_TRUCK': 'SKU','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)

Maestro_SKU.SKU = Maestro_SKU.SKU.astype(str)
Maestro_SKU.SKU = Maestro_SKU.SKU.str.split(".", n=1, expand=True)

Maestro_SKU_KOE = Maestro_SKU[["SKU","SKU KOE","SKU_SAP"]]
Maestro_SKU_KOE = Maestro_SKU_KOE.drop_duplicates(['SKU KOE'])
Maestro_SKU_KOE.rename(columns = {'SKU KOE':'Sku_KO'} ,inplace=True)
Maestro_SKU_KOA = Maestro_SKU[["SKU","Sku_KOA","SKU_SAP"]]
Maestro_SKU_KOA = Maestro_SKU_KOA.drop_duplicates(['Sku_KOA'])
Maestro_SKU_KOA.rename(columns = {'Sku_KOA':'Sku_KO'} ,inplace=True)

Maestro_SKU_KO = Maestro_SKU_KOA.append(Maestro_SKU_KOE)
Maestro_SKU_KO.SKU = Maestro_SKU_KO.SKU.astype(str)
Maestro_SKU_KO.Sku_KO = Maestro_SKU_KO.Sku_KO.astype(str)
Maestro_SKU_KO.SKU_SAP = Maestro_SKU_KO.SKU_SAP.astype(str)
Maestro_SKU_KO = Maestro_SKU_KOA.append(Maestro_SKU_KOE)
Maestro_SKU_KO = Maestro_SKU_KO.drop_duplicates(['Sku_KO'])
Maestro_SKU_KO = Maestro_SKU_KO.dropna(subset=['Sku_KO','SKU','SKU_SAP'])

# E2 C3.2 SKU cch
SO = pd.merge(SO,Maestro_SKU_KO ,on=["Sku_KO"],how="outer")
SO = SO.dropna(subset=['Sku_KO','CD','Seg','Fecha_f','Hl'])
# E2 C4 Segmento
SO["Seg"] = SO.Seg.str.replace(r" ","")
SO["Seg"] = SO.Seg.str.lower()
SO["Seg"] = SO.Seg.str.replace(r"-","")
SO["Seg"] = SO.Seg.str.replace(r"nacionales","")
SO["Seg"] = SO.Seg.str.replace(r"regionales","")
# E2 C5 Segmento
SO.C_Cli = SO.C_Cli.astype(str)
Titulo = SO["C_Cli"].str.split(".", n=2, expand=True)
SO['C_Cli'] =  Titulo[0].astype(str)# + Titulo[1].astype(str) + Titulo[2].astype(str)

### Mestra mes
#SO_m = SO[SO["Mes"]==8]
#SO_m = SO_m[["Fecha_f", "CD", "C_Cli", "Seg", "Sku_KO", "CJ", "Hl", "SKU", "SKU_SAP" ]]
#SO_m.to_csv("D:/Reporting/ETL/01 SELL/Sell Out/SO_cierre_m.txt")

# E3 Exportar resultados #################
SO = SO[["Fecha_f", "A_Mes", "CD", "C_Cli" , "Seg", "Sku_KO","SKU", "SKU_SAP", "CJ", "Hl","Bottler" ]]
SO = SO.dropna(subset=['SKU'])

# E3.1 Base Año mes
Mes = SO.groupby(by = ["A_Mes", "CD","Seg", "SKU"])["Hl"].sum().reset_index()
Mes = pd.DataFrame(Mes)
Mes.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\01 SELL\Sell Out\Mensual.txt")

# E3.2 Base completa detalle cliente, sku, dia
SO = SO[["Fecha_f", "CD", "C_Cli","Seg", "Sku_KO","SKU", "SKU_SAP", "CJ", "Hl","Bottler"]]
SO.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\01 SELL\Sell Out\SO_cierre.txt")

##### Resumen para combinar con con base de recortes
#SO.C_Cli = SO.C_Cli.astype(str)
##SO['Fecha_t'] = SO.Fecha_f.astype(str)
#SO.Sku_KO = SO.Sku_KO.astype(str)
#Titulo = SO["C_Cli"].str.split(".", n=2, expand=True)
#SO['C_Cli'] =  Titulo[0].astype(str)# + Titulo[1].astype(str) + Titulo[2].astype(str)
#Titulo = SO["Sku_KO"].str.split(".", n=2, expand=True)

#SO['Sku_KO'] =  Titulo[0].astype(str)# + Titulo[1].astype(str) + Titulo[2].astype(str)
#SO["Fecha_t"] = SO.Fecha_t.str.replace(r"-","")

#SO['ID'] = SO.Fecha_t +"-"+ SO.C_Cli + "-"+ SO.Sku_KO
#SOr = SO[["ID", "CJ", "Hl"]]
#SOr = SOr.groupby(by = ["ID"])["CJ", "Hl"].sum().reset_index()
#SOr = pd.DataFrame(SOr)
#SOr



In [2]:
#### RECORTES
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\001 Sharepoint Jorge')
Sh_j = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep='\t')#,dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')
    Sh_j.append(data)
    
Sh_j = pd.concat(Sh_j,ignore_index=True)
Sh_j = Sh_j[Sh_j["cga_tip_cga"]=="LOGISTICA VFILLRATE_SERVICE_LEVEL"]
Sh_j = Sh_j.dropna(subset=['log_anno_mes_dia_ent'])
Sh_j = Sh_j[['log_anno_mes_dia_ent', 'log_cod_cliente','log_sku_producto','log_seg_mrkt','log_can_cjs_ven','log_tip_rec','log_cod_cd']]

#### Cajas recorte
Sh_j.log_can_cjs_ven = Sh_j.log_can_cjs_ven.astype(float)
Sh_j.log_can_cjs_ven = Sh_j.log_can_cjs_ven.round(3)

###### MAESTRAS PEPO ####
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Maestras')
Maestro_SKU = pd.read_excel('Nueva mastra SKU_PEPO.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','SKU_TRUCK': 'SKU','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)
Maestro_SKU.SKU = Maestro_SKU.SKU.astype(str)
Maestro_SKU.SKU = Maestro_SKU.SKU.str.split(".", n=1, expand=True)
Maestro_SKU_KOE = Maestro_SKU[["SKU","SKU KOE","SKU_SAP"]]
Maestro_SKU_KOE = Maestro_SKU_KOE.drop_duplicates(['SKU KOE'])
Maestro_SKU_KOE.rename(columns = {'SKU KOE':'Sku_KO'} ,inplace=True)
Maestro_SKU_KOA = Maestro_SKU[["SKU","Sku_KOA","SKU_SAP"]]
Maestro_SKU_KOA = Maestro_SKU_KOA.drop_duplicates(['Sku_KOA'])
Maestro_SKU_KOA.rename(columns = {'Sku_KOA':'Sku_KO'} ,inplace=True)
Maestro_SKU_KO = Maestro_SKU_KOA.append(Maestro_SKU_KOE)
Maestro_SKU_KO.SKU = Maestro_SKU_KO.SKU.astype(str)
Maestro_SKU_KO.Sku_KO = Maestro_SKU_KO.Sku_KO.astype(str)
Maestro_SKU_KO.SKU_SAP = Maestro_SKU_KO.SKU_SAP.astype(str)
Maestro_SKU_KO = Maestro_SKU_KOA.append(Maestro_SKU_KOE)
Maestro_SKU_KO = Maestro_SKU_KO.drop_duplicates(['Sku_KO'])
Maestro_SKU_KO = Maestro_SKU_KO.dropna(subset=['Sku_KO','SKU','SKU_SAP'])
FL_Sh_c2 = Sh_j 
FL_Sh_c2.rename(columns = {'log_sku_producto': 'Sku_KO'} ,inplace=True)
FL_Sh_c2 = pd.merge(FL_Sh_c2,Maestro_SKU_KO,on=["Sku_KO"],how="left")
FL_Sh_c2 = FL_Sh_c2.dropna(subset=['SKU'])
FL_Sh_c2 = FL_Sh_c2[FL_Sh_c2["SKU"]!="nan"]

FL_Sh_c2.log_can_cjs_ven = FL_Sh_c2.log_can_cjs_ven.astype(float)
FL_Sh_c2["log_seg_mrkt"] = FL_Sh_c2.log_seg_mrkt.str.replace(r"SUPERMERCADO-HIPERMERCADO","supermercados")
FL_Sh_c2["log_seg_mrkt"] = FL_Sh_c2.log_seg_mrkt.str.replace(r"TRADICIONAL","mercadohogar")
FL_Sh_c2["log_seg_mrkt"] = FL_Sh_c2.log_seg_mrkt.str.replace(r"MAYORISTA","mayoristas")
FL_Sh_c2["log_seg_mrkt"] = FL_Sh_c2.log_seg_mrkt.str.replace(r"COMER Y BEBER","mercadorefrigerado")

os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Maestras')
Maestro_CD = pd.read_excel('Maestra de clientes.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_CD.rename(columns = {'CD_ID_Num': 'ID_Num','CP_SO_Cierre': 'SO_Cierre'} ,inplace=True)
Maestro_CD = Maestro_CD[["ID_Num", "SO_Cierre" ]]
Maestro_CD = Maestro_CD.dropna(subset=['ID_Num'])

FL_Sh_c2.rename(columns = {'log_cod_cd': 'ID_Num'} ,inplace=True)
FL_Sh_c2.ID_Num = FL_Sh_c2.ID_Num.astype(str)
t = FL_Sh_c2.ID_Num.str.split(".", n=1, expand=True)
FL_Sh_c2["ID_Num"] = t[0]

FL_Sh_c4 = pd.merge(FL_Sh_c2,Maestro_CD,on=["ID_Num"],how="left")
FL_Sh_c4["Bottler"] = "Embonor"

####### Maestro de Clientes ########
M_Cli = FL_Sh_c4[["log_cod_cliente", "log_seg_mrkt", "SO_Cierre","ID_Num", "Bottler"]]
M_Cli = M_Cli.drop_duplicates(subset=["log_cod_cliente"])
M_Cli.rename(columns = {'log_cod_cliente': 'C_Cli','log_seg_mrkt': 'Seg', 'SO_Cierre':'CD'} ,inplace=True)

M_Cli_SO = SO[["C_Cli", "Seg", "CD", "Bottler"]]
M_Cli_SO = M_Cli_SO.drop_duplicates(subset=["C_Cli"])
M_Cli = M_Cli.append(M_Cli_SO)
M_Cli = M_Cli.drop_duplicates(subset=["C_Cli"])

#FL_Sh_c2["ID"] = FL_Sh_c2.log_anno_mes_dia_ent.astype(str) +"-"+ FL_Sh_c2.log_cod_cliente.astype(str) +"-"+ FL_Sh_c2.Sku_KO.astype(str)
#FL_Sh_c3 = FL_Sh_c2[["ID", "log_can_cjs_ven"]]
#CP = FL_Sh_c3.append(SOr)
#CP = CP.drop_duplicates(subset=["ID"])
#CP = CP[["ID"]]
#DATA = pd.merge(CP,FL_Sh_c3,on=["ID"],how="left")
#DATA = pd.merge(DATA,SOr,on=["ID"],how="left")

M_Cli.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\04 Service Level\01 Recores Share KOE\Clientes.txt")
FL_Sh_c4.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\04 Service Level\01 Recores Share KOE\Recortes_KO.txt")